## Dependencies

In [1]:
import pandas as pd
import statsmodels.api as sm
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

## Data Extraction

In [2]:
# Load Excel file
merged_path = os.path.join("..", "data", "indonesia", "merged_indonesia.xlsx")
df = pd.read_excel(merged_path)

# Clean column names
df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]

# Rename for clarity
df = df.rename(columns={
    "cpi_data": "inflation_rate",
    "manufacturing_(%_age_of_gdp)": "manufacturing_output",
    "trade_balance_(%_age_of_gdp)": "trade_deficit",
    "tariff_rate": "tariff"
})

# Preview the dataframe
print(df)


   year  inflation_rate  manufacturing_output  trade_deficit  tariff
0  2014             6.4                0.2108        -0.0075  0.0250
1  2015             6.4                0.2099         0.0038  0.0210
2  2016             3.5                0.2052         0.0076  0.0264
3  2017             3.8                0.2016         0.0100  0.0206
4  2018             3.2                0.1986        -0.0107  0.0200
5  2019             2.8                0.1970        -0.0044  0.0200
6  2020             2.0                0.1987         0.0169  0.0204
7  2021             1.6                0.1924         0.0263  0.0183
8  2022             4.2                0.1834         0.0354  0.0000
9  2023             3.7                0.1867         0.0218  0.0000


## Defining The Variables

In [3]:
# Independent variable (X)
X = df[["tariff"]]

# Dependent variables (Y)
y_inflation = df["inflation_rate"]
y_manufacturing = df["manufacturing_output"]
y_trade_balance = df["trade_deficit"]


## Statistical Analysis

In [4]:
model_inflation = LinearRegression().fit(X, y_inflation)
model_manufacturing = LinearRegression().fit(X, y_manufacturing)
model_trade_balance = LinearRegression().fit(X, y_trade_balance)


In [5]:
print("Inflation Model:")
print("Coefficient:", model_inflation.coef_)
print("Intercept:", model_inflation.intercept_)

print("\nManufacturing Output Model:")
print("Coefficient:", model_manufacturing.coef_)
print("Intercept:", model_manufacturing.intercept_)

print("\nTrade Balance Model:")
print("Coefficient:", model_trade_balance.coef_)
print("Intercept:", model_trade_balance.intercept_)


Inflation Model:
Coefficient: [12.10708596]
Intercept: 3.552121334057128

Manufacturing Output Model:
Coefficient: [0.8403454]
Intercept: 0.18400126954842644

Trade Balance Model:
Coefficient: [-1.11387469]
Intercept: 0.029045228342829944


In [6]:
X_sm = sm.add_constant(X)  # Adds intercept term manually

model_inf = sm.OLS(y_inflation, X_sm).fit()

model_manu = sm.OLS(y_manufacturing, X_sm).fit()

model_trade = sm.OLS(y_trade_balance, X_sm).fit()


In [7]:
def save_ols_summary(x, y, data, filename):
    X = sm.add_constant(data[[x]])
    y_vals = data[y]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = sm.OLS(y_vals, X).fit()

    summary_str = model.summary().as_text()

    output_path = os.path.join("..", "output", "indonesia", filename)
    with open(output_path, "w") as f:
        f.write(summary_str)

In [8]:
save_ols_summary("tariff", "inflation_rate", df, "inflation_ols.txt")
save_ols_summary("tariff", "manufacturing_output", df, "manufacturing_ols.txt")
save_ols_summary("tariff", "trade_deficit", df, "trade_ols.txt")

## Saving Plots

In [9]:
def regression_plot_save(x, y, data, title, color, filename):
    X = sm.add_constant(data[[x]])
    y_vals = data[y]

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model = sm.OLS(y_vals, X).fit()
        pred = model.predict(X)

    r_squared = model.rsquared
    p_val = model.pvalues[1]
    slope = model.params[1]

    plt.figure()
    sns.regplot(x=x, y=y, data=data, ci=None, color=color)
    plt.title(title)
    plt.xlabel("Tariff Rate (%)")
    plt.ylabel(y)

    # Add textbox
    textstr = f"$R^2$ = {r_squared:.3f}\nSlope = {slope:.3f}\np = {p_val:.3f}"
    plt.text(0.05, 0.95, textstr, transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='top',
             bbox=dict(boxstyle='round,pad=0.4', facecolor='white', alpha=0.5))

    # Save the figure
    output_path = os.path.join("..", "output", "indonesia", filename)
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()  # Close to avoid displaying.



In [10]:
regression_plot_save("tariff", "inflation_rate", df, "Tariff vs Inflation Rate (CPI)", "blue", "inflation_plot.png")
regression_plot_save("tariff", "manufacturing_output", df, "Tariff vs Manufacturing Output", "green", "manufacturing_plot.png")
regression_plot_save("tariff", "trade_deficit", df, "Tariff vs Trade Deficit", "red", "trade_plot.png")